#### GISC 420 T1 2022
# **LAB ASSIGNMENT 2**
Corvin Idler, ID 300598312

## **Code already provided**


In [ ]:
# You need to run this cell to get things setup
%matplotlib inline
import matplotlib
import matplotlib.pyplot as pyplot
import geopandas
import shapely
import math

Definition of lon_lat_at_bearing_and_distance() and geodesic_circle() function

In [ ]:
# returns a lon-lat point (in degrees) given
# a starting lon-lat and bearing and distance **(all in radians)**
# equation is from
# https://stackoverflow.com/questions/7222382/get-lat-long-given-current-point-distance-and-bearing
def lon_lat_at_bearing_and_distance(p, b, d):
    lon1 = p[0]
    lat1 = p[1]
    lat2 = math.asin(math.sin(lat1) * math.cos(d) + 
                     math.cos(lat1) * math.sin(d) * math.cos(b))
    lon2 = lon1 + math.atan2(math.sin(b) * math.sin(d) * math.cos(lat1), 
                             math.cos(d) - math.sin(lat1) * math.sin(lat1))
    # convert result to degrees before returning
    return (math.degrees(lon2), math.degrees(lat2))


# makes a shapely.geometry in lon-lat coordinates
# based on a provide centre point in lon-lat and 
# radius expressed **in degrees**
def geodesic_circle(p=(0,0), rd=2):
    # extract the numbers and convert to radians
    # assume p is a (lon, lat) tuple in degrees
    ll = (math.radians(p[0]), math.radians(p[1]))
    # convert rd to degrees
    rr = math.radians(rd) 
    
    # empty list for the resulting points
    pts = []
    # 360 in 1 degree steps
    # each time get a point at the distance rr away in that direction
    for bearing in range(360):
        pts.append(lon_lat_at_bearing_and_distance(ll, math.radians(bearing), rr))
    return shapely.geometry.Polygon(pts)


Function definition of make_gdf_from_circles()

In [ ]:
def make_gdf_from_circles(circles):
    gs = geopandas.GeoSeries(circles)
    gdf = geopandas.GeoDataFrame(geometry=gs)
    gdf.crs = "EPSG:4326" # tell it we are using simple lon-lat
    return gdf

Set up a base map

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

Load file with equally spaced grid centres (`cc.csv`). Using the filepath needed to make it run from within the binder image

In [ ]:
# The open() function opens the specified file for reading into a list of lines
with open("labs/sequences-and-iteration/cc.csv") as file:
    data = file.readlines()

## **As from here my own work**
## **First part of the assignement**

In [ ]:
# first make an empty list for the circles
# call it something appropriate
circle_list = []

# loop through the data read from the file using a for loop
# you can skip the first line with a slice operation
for line in data[1:]:

# inside the loop, with each line of the data:
# first, remove the last character (the newline character) (this is another slice operation)
    temp = line[:-1]

# second, split the line into two strings at the ',' separator (look back to the available string functions)
    long, lat = temp.split(',')

# now you should have a list of two items, the lon and lat, as strings
# so, third, you need to convert these to float values and then pass them into
# the geodesic_circle function as the p parameter
# finally, append the resulting geodesic_circle to the list of circles
    circle_list.append(geodesic_circle(p=(float(long),float(lat))))

In [ ]:
#make a GeoDataFrame from your list of circles
gdf = make_gdf_from_circles(circle_list)

In [ ]:
#Plotting the results
fig = pyplot.figure(figsize=(12,6))
base = fig.add_subplot(111)

world.plot(ax=base, facecolor='lightgrey')
gdf.plot(ax=base, facecolor='r', alpha=0.5)

## **The second part of the assignment**

In [ ]:
# Make an empty list for the circles
new_circle_list = []

# Use nested for loops and the range operation to make a series of circles
# arranged in a longitude - latitude grid, appending them to the list each time
for long in range(-175, 175, 15):
    for lat in range(-75, 76, 15):
        new_circle_list.append(geodesic_circle(p=(float(long),float(lat))))

# then use make_gdf_from_circles() to make GeoDataFrame from your list of circles
new_gdf = make_gdf_from_circles(new_circle_list)

# then use the code you used before to make a map
fig = pyplot.figure(figsize=(12,6))
base = fig.add_subplot(111)

world.plot(ax=base, facecolor='lightgrey')
new_gdf.plot(ax=base, facecolor='r', alpha=0.5)

## Commentary
No major coding problems. Fiddled around a bit with the range definition in the second part of the assignement, to make it visually appealing (as I didn't want dots that are hovering over empty space without a landmass).

Main difference between the map in the first part of the assignement and the one in the second part of the assignement is obviously that the grid in the first part is equally spaced on a 3D sphere. That means particularly the convergence of the meridians at the poles is taken into account.

The spacing of the circles in the second map has a fixed step size regardless of the latitude of the coordinates. That means the circles appear more or less equally spaced in the projection that was used, but are certainly not equally spaced on a 3D sphere (high latitudes vs. low latitudes).

## Comments
All good. 

For the string processing, I suppose the prompt is to use a slice to remove the newline characters. An alternative is `line = line.strip()` Note that you don't really need a `temp` you can just do 

    lon, lat = line.strip().split(",")
    
if you really want to go for it you can even do 

    lonlat = [float(s) for s in line.strip().split(",")]
    
Of course it's all a bit academic given that reading the data using `pandas` is a better idea anyway!

One option for avoiding the dateline and poles would be

    circ_r = 2
    step = int(circ_r * 2)
    for x in range(-180 + circle_r, 181 - circ_r, step):
        ...
    
or similar. It might not be failsafe (almost certainly not) but it would avoid a lot of the problem coordinates.

**A**